## TODOs

- put on git
- DQN, could vary EPSILON_RANGE as well
- if done, r = 0 !!!!!, anstatt r_t? als parameter flag
- append code with full model -> "File, Download as Python"
- Computationally intensive! / Use Spark on AWS
    - https://eu-central-1.console.aws.amazon.com/ec2/v2/home?region=eu-central-1#LaunchInstanceWizard:
    - http://blog.insightdatalabs.com/spark-cluster-step-by-step/
    - http://blog.insightdatalabs.com/jupyter-on-apache-spark-step-by-step/
    - http://stackoverflow.com/questions/37327021/using-jupyter-notebook-on-spark-on-emr

# Reinforcement Learning: Implementing and Comparing three Algorithms

## Why Reinforcement Learning As My Capstone?

### Motivation

I have a background in supervised learning mostly, a bit also unsupervised, and I've been applying these techniques to hands-on business use cases for 10 years now.

When I applied for the machine learning engineer nanodegree, I wanted to deepen my knowledge in areas known to me, and also discover new areas.

Both objectives were met during the course. The latter came into play with the Smartcab task, and I was struck. This was new. It combined two areas where I see a steep learning curve for me as well as a lot of interesting applications in business life:

- First, Artifical Neural Nets as the main computing block of learning. During my studies, networks were not as much on the screen as today. During this project, I will thus need to dive into this topic and gain insights which will drive my interest and career onwards. I see networks as a generic computing block, helpful in many areas of prediction.

- Second, Reinforcement Learning. Markov Decision Processes are just a lot of fun and a real challenge for a greenhorn brain. Still, there are components of supervised learning and statistics in general, which I can apply hereby. Lastly, I am interested in robotics and learning in general, i.e. getting (sensory) data, transform it, predict, and see how the reward or punishment gets in. Human learning is fascinating, as is machine learning.

- Third, Cloud Computing. During the computation of quantile estimates for my master thesis in 2009, I remember my Macbook running during days and getting so hot that I had to put Ice underneath it. Here, I want to get aquainted with the resources available on the cloud to do heavy computing. I will hook onto Amazon Web Services EC2 Instances with GPUs in order to compute the Keras/TensorFlow parts of my Capstone. 

### Outline

- I will try to train an toy, Atari-based AI agent on 1D input data. 

- I will not touch 2D visual inputs, since I found that convolutions are a separate topic block whick easily can be implemented as an extension to the presented building blocks. Besides that, one could also convert 2D to input 1D beforehand and proceed with the presented blocks.

- The models will consist of fully-connected layers, activation functions and output layers in the appropriate form.

- In order to train the model, I will benchmark three reinforcement learning algorithms:
    - Deep Q Learning
    - Advantage Actor-Critic Policy Gradients
    - Q Actor-Critic Policy Gradients

Deep Q Learning is the natural entry point, since I can use my so-far knowledge of standard Q Learning and go one step further with it. Besides that, it only uses one model, compared with Policy Gradients, where two models come into play. I will develop the algorithm step-by-step, since it was the process I had to go through anyway to understand the logic. 

Policy Gradients will not be developed step-by-step, but recycling many building blocks of Deep Q Learning as well as the methodology.

Benchmarking will be starting with brute-force parametrization combinatorics in order to hopefully find working models.

Since this is expected to be computationally intensive, I will shift computation to AWS EC2 cloud.

### Credits and Thanks

- Neural Networks
    - Andrew Trask https://iamtrask.github.io
    - Sebastian Raschka http://sebastianraschka.com/Articles/2015_singlelayer_neurons.html
- Deep Q Learning
    - Deep Mind: Playing Atari with Deep Reinforcement Learning https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf
    - Tambet Matiisen https://www.nervanasys.com/demystifying-deep-reinforcement-learning/ https://github.com/tambetm/simple_dqn/blob/master/src/replay_memory.py
    - Eder Santana http://edersantana.github.io/articles/keras_rl/
    - Ben Lau https://yanpanlau.github.io/2016/07/10/FlappyBird-Keras.html
- Policy Gradients
    - Andrej Karpathy http://karpathy.github.io/2016/05/31/rl/
    - Denny Britz https://github.com/dennybritz/reinforcement-learning/tree/master/PolicyGradient/
    - Open AI Gym https://gym.openai.com/docs/rl#policy-gradients
    - David Silver https://www.youtube.com/watch?v=KHZVXao4qXs
- Keras
    - Francois Chollet https://keras.io https://github.com/fchollet/keras/tree/master/examples 
https://github.com/fchollet/keras/issues/3062
- Amazon Web Services EC2
    - The Amazon Tutorials https://aws.amazon.com/de/ec2/
    - Jie Yang http://yangjie.me/2015/08/26/Run-Jupyter-Notebook-Server-on-AWS-EC2/
    
### Main Dependencies

We are working with Elon Musk's Open AI Gym (https://gym.openai.com/) as a training environment for our AI agent.

Lunar Lander environment (https://gym.openai.com/envs/LunarLander-v2) is particularily appealing to me. It is based on box2d, which simulates real life physics. Charming! And, with its 1D input state vector, it is a first step for creating and tuning an AI agent, before proceeding with convolution preprocessing of 2D inputs.

The environment home page says the following:

*"Landing pad is always at coordinates (0,0). Coordinates are the first two numbers in state vector. Reward for moving from the top of the screen to landing pad and zero speed is about 100..140 points. If lander moves away from landing pad it loses reward back. Episode finishes if the lander crashes or comes to rest, receiving additional -100 or +100 points. Each leg ground contact is +10. Firing main engine is -0.3 points each step. Solved is 200 points. Landing outside landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt. Four discrete actions available: do nothing, fire left orientation engine, fire main engine, fire right orientation engine."*

Model building and training is done with Keras (https://keras.io). This modular, minimalist library makes ANN life as easy as it can get, and in plus runs on both Theano and Tensorflow backends.

# Reinforcement Learning Part 1: Deep Q Learning

## The Starting Point: Evaluating Bellman Equations From Data (Q-Learning)

Q learning is about revisiting states. We are in a specific state s at time t, and because the state space is sufficiently small, we might discover that the agent has already been in s before. It therefore has made an experience for s by taking an action, and collecting a reward or punishment. All this is stored in the agent's "memory" (possible a dictionary of dictionaries, main keys being the states, values being the actions (keys) and their values). We want the agent to take advantage of this "memory": We look up the expected lifetime rewards per each possible action in s (a.k.a. action-value function, q values), select the maximum q value, and execute the chosen action. 

We now have fresh evidence about the consequence of a specific action in a specific state: We know the initial state *s_t*, we know the selected action *a_t*, we know the reward *r_t*, and we know the new state this all lead to, s_t1. This knowledge we now use to update the agent's memory: We calculate a new q value for *s_t* by taking the observed reward *r_t*, and adding to it the discounted maximum q value for *s_t1*. The difference to the old q value is the new q value for action *a_t* in state *s_t*.

### Why Q-Learning Often Does Not Work: Exploding State Spaces

Revisiting states is often not possible, even in the long run, because there are simply too many combinations of relevant inputs which constitute a state. Just think a small number of inputs, each input being a floating number with 4 digits. Even this small setting is creating a large amount of combinations: the state space explodes. Revisiting states is very unlikely, we will need a huge number of trials to generate memory updates. As a consequence, learning is slow, or even not happening.

In order to illustrate the point, we are going to set up a basic q-learning algorithm:

#### Namespace

In [1]:
import numpy as np
import gym
from itertools import count
from sys import stdout

#### Global Seed

In [2]:
np.random.seed(0)

#### Set Hyperparameters

- *NUM_EPISODE* denotes the maximum number of episodes an epoch will embrace
- *GAMMA* is the factor by which future expected rewards are discounted
- *ALPHA* is the learning rate TODO
- *Q_TABLE* is the agent's memory. The states are the keys, and dictionaries of actions and their respective values are the values
- *VALUE_INIT* is the initial value for the actions of a state, once the state is visited the first time. It is set to zero.

In [3]:
NUM_EPISODE = 100
GAMMA = 0.99
ALPHA = 0.1
Q_TABLE = {}
VALUE_INIT = 0

#### Prepare Environment

We create an instance (*ENV*) of the Lunar lander environment. Its input dimensions *NUM_INPUT* are obtained by resetting the environment, and the actions by the environment method *action_space*.

In [4]:
ENV = gym.make("LunarLander-v2")
NUM_INPUT = ENV.reset().shape[0]
NUM_ACTION = ENV.action_space.n
ACTIONS = np.arange(0, NUM_ACTION)

[2016-11-07 21:58:39,070] Making new env: LunarLander-v2


#### Build Training Epoch

We are going to create the main building block of this exercise: The training block for one epoch.

In [5]:
def train_ql(render=False):
    # Init epoch stats
    stats = {}
    
    # Init counter for how many times we revisited states
    revisiting_states = 0 

    # Play through episodes
    for episode in range(NUM_EPISODE):
        # Init episode stats
        stats[episode] = [0]*3
        
        # Init observations
        x_t = ENV.reset() 
        # Take only the current observation as state, in order to keep the state space as small as possible
        s_t = tuple(x_t) 
        done = False

        # Start an episode
        for step in count():
            
            # Reset the environment for a new gaming step
            if render: ENV.render()
            
            # Look up the action with the highest value at s_t, as well as its value
            a_t, q, r_s  = best_action(s_t)

            # Observe after action a_t
            x_t1, r_t, done, info = ENV.step(a_t) 
            # Again, take only the current observation
            s_t1 = tuple(x_t1) 
            
            # Look up the action with the highest value at s_t1, as well as its value
            a_t1, Q_sa, _ = best_action(s_t1) 
            
            # Look up the action with the highest value at s_t1, as well as its value
            Q_TABLE[s_t][a_t] = q + ALPHA * (r_t + GAMMA * Q_sa - q) 
            
            # Update statistics: Sum up rewards, count steps, successes, solved
            stats[episode][0] += r_t
            if r_t >= 100: stats[episode][1] += 1
            if r_t >= 200: stats[episode][2] += 1
            
            # Visualize
            stdout.write("\rStep {} @ Episode {}/{}: Reward {}, Success {}, Solved {}".format(
                       step, episode+1, NUM_EPISODE, stats[episode][0], stats[episode][1], stats[episode][2], end="")) 
            
            # Start a new episode after crashing or deadlock
            if done or step > 10000: break
            
            # Update states
            s_t = s_t1
            revisiting_states += r_s
            
    # Visualize revisiting states issue
    print "\rQ Table size {}, # Revisited States {}".format(len(Q_TABLE), revisiting_states)
    
    return stats
    
    
# Create helper function to initialize and query Q-table
def best_action(state):
    # Create init scenario for table queries at t and t1
    if state not in Q_TABLE or sum(Q_TABLE[state].values()) == 0: 
        
        # Bookkeeping: Init revisiting states counter
        revisit_state = 0 
        
        # Init q function
        q_function = {} 
        for A in ACTIONS: q_function[A] = VALUE_INIT
        Q_TABLE[state] = q_function 
        
        # Do random action
        action = np.random.choice(ACTIONS, 1)[0] 
    else: 
        revisit_state = 1
        
        # Select action according to max q
        action = max(Q_TABLE[state], key=Q_TABLE[state].get) 
    
    # Get q value for action selected
    q = Q_TABLE[state][action] 
    
    return action, q, revisit_state

#### Train Model

In [6]:
stats = train_ql() # Train model / Play epoch
print "Maximum Reward {}, Successful Episodes {}, Solved Episodes {}".format\
                    ( max([ v[0] for v in stats.values() ])\
                     , sum([ v[1] for v in stats.values() ])\
                     , sum([ v[2] for v in stats.values() ]))

Q Table size 18883, # Revisited States 0
Maximum Reward -11.8889336317, Successful Episodes 5, Solved Episodes 0


## What to Do? Do Not Update the Q Function, But the Q  Function Estimator: Deep Q Learning

In this situation, we replace the "revisiting states" by a function approximator: 

We let a Artifical Neural Net (ANN) estimate the q function for the state s the agent is visiting at time t. 

Once we performed the action based on the maximum of the q function (just the action with the highest expected lifetime reward at time t), we know the reward, and the subsequent state. 

Based on this, we are able to update the agent's memory. But this time, we do not update the q function directly. Instead, we are updating the ANN, which means that we are updating the weights used in the ANN. And that is how exactly:

- At time t, we already know the estimation of the q function for state s_t: We used it to pick an action a_t accordingly. Read again: this is the **estimation** of the q function.

- After action a_t, we know *s_t*, *a_t*, *r_t* and *s_t1*. This allows us to update the q function, **but only for the action taken**. We take r_t, and add to it the discounted expected lifetime reward, in other words we let the ANN estimate the q function for state s_t1. 

- For the action taken, we can update the q value now. All the other actions are not performed, we do not know about the reward, or a subsequent state *s_t1*. So, we cannot learn for those. This updated q function is the **target**.

- We feed the error, which is the difference between the estimation and the target.

- We backpropagate the error through the network, such that the weights are updated.

Next time we estimate the q function for another state s, we have updated weights

### 0) Understand the Gist of ANNs

Firstly, I wanted to understand how a neural network works. Andrew Trask's excellent toy examples helped me understand it. I create a network with one hidden layer, and the output layer, both sigmoid activated, which returns probablities for each of the outputs.

I will use a sigmoid activation function due to the easy calculation of the derivative and the widespread application in neural nets.

Except for numpy, there are no other dependencies.

In the procedure, the magic happens with *layer_2_weighted_loss*: Each output loss is multiplied by the slope / gradient of the predicted value on the sigmoid curve.

In order to get a feeling for changes invoked by changing parametrization, I will alter the learning rate *ALPHA* as well as the number of neurons in the hidden layer *NUM_HIDDEN_NEURON* within an arbitrary range.

#### Set Hyperparameters

In [7]:
ALPHA_RANGE = [10**-6, 10**-4, 10**-2, 1, 10**2, 10**4, 10**6] # Constant over one epoch
NUM_HIDDEN_NEURON_RANGE = [2**i for i in np.arange(11)] # Constant over one epoch

#### Create A Basic ANN

In [8]:
# Create the helper functions 

# Create the sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Create the sigmoid function's derivative
def sigmoid_derivative(x):
    return x * (1 - x)


# Initialize weights with the correct dimensionality to fit in the input layer
X = np.random.randint(2, size=(4, 3))
y = np.random.randint(2, size=(4, 1))

# Test the hidden dimensions
for NUM_HIDDEN_NEURON in NUM_HIDDEN_NEURON_RANGE: 
    
    # Initialize 1st set of weights
    W1 = np.random.rand(X.shape[1], NUM_HIDDEN_NEURON) 

    # Initialize 2nd set of weights
    W2 = np.random.rand(NUM_HIDDEN_NEURON, y.shape[1]) 
    
    # Test the alphas
    for ALPHA in ALPHA_RANGE: 
        
        for episode in range(NUM_EPISODE):
            # Forward propagate
            
            # Initialize hidden layer (fully connected)
            layer_1 = np.dot(X, W1)
            
            # Apply sigmoid activation
            layer_1 = sigmoid(layer_1) 
            
            # Initialize output layer(fully connected)
            layer_2 = np.dot(layer_1, W2) 
            
            # Apply sigmoid activation 
            layer_2 = sigmoid(layer_2) 

            # Calculate loss
            layer_2_loss = y - layer_2 

            ''' Apply SGD to the loss: the more certain the estimate, the less weighted it will get: 
                The gradient at the extremes is smaller than in the middle
            '''
            layer_2_weighted_loss = layer_2_loss * sigmoid_derivative(layer_2) # element-wise multiplication!
            
            # Backpropagate
            
            # Compute the effect of the hidden layer to the weighted loss
            layer_1_loss = np.dot(layer_2_weighted_loss, W2.T) 

            # Apply SGD
            layer_1_weighted_loss = layer_1_loss * sigmoid_derivative(layer_1) 
                
            # Update weights
            W2 += ALPHA * np.dot(layer_1.T, layer_2_weighted_loss)
            W1 += ALPHA * np.dot(X.T, layer_1_weighted_loss)
            
            # Visualize
            if episode == NUM_EPISODE - 1: print "Hidden Size {}, alpha {}: Avg loss {}".format(
                                                '%4s' % NUM_HIDDEN_NEURON, \
                                                '%7s' % ALPHA, \
                                                '%14s' % np.mean(np.abs(layer_2_loss)))

Hidden Size    1, alpha   1e-06: Avg loss 0.495156182726
Hidden Size    1, alpha  0.0001: Avg loss 0.495155860436
Hidden Size    1, alpha    0.01: Avg loss 0.495157688542
Hidden Size    1, alpha       1: Avg loss 0.484806495108
Hidden Size    1, alpha     100: Avg loss 0.499999956387
Hidden Size    1, alpha   10000: Avg loss 0.499998267408
Hidden Size    1, alpha 1000000: Avg loss            0.5
Hidden Size    2, alpha   1e-06: Avg loss 0.491235706692
Hidden Size    2, alpha  0.0001: Avg loss 0.491233741973
Hidden Size    2, alpha    0.01: Avg loss 0.491173219296
Hidden Size    2, alpha       1: Avg loss 0.377296429514
Hidden Size    2, alpha     100: Avg loss 0.499999072308
Hidden Size    2, alpha   10000: Avg loss            0.5
Hidden Size    2, alpha 1000000: Avg loss            0.5
Hidden Size    4, alpha   1e-06: Avg loss 0.499565230047
Hidden Size    4, alpha  0.0001: Avg loss 0.499558683046
Hidden Size    4, alpha    0.01: Avg loss 0.498832815676
Hidden Size    4, alpha       1

#### Observations

Clearly visible, the parametrization of the network has an impaction upon the network performance. 

After looking at the configurations, I'd probably set the hidden size to 8, and alpha around 100, due to a low average loss. Here is an example i got of one of the runs:

*Hidden Size    8, alpha      100: Final avg loss   0.175510028718*

### 1) Deep Q Learning from SIngle Current Observations

In order to implement the estimator, we now change the training epoch function. We first get rid of the lookup table *Q_TABLE* and the table's Q value initialization *Q_INIT*. In plus, we do not need the learning rate *ALPHA* anymore. In stead, we implement the above-standing routine 1) to 5). 

The core building block of compute will be a "shallow" ANN with one hidden layer built with Keras. Why shallow? Building a model is connected to a steep learning curve for me. So, I will try to keep parametrization combinatorics at bay during this steep personal ascent.

#### Namespace (Extension)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam 

Using TensorFlow backend.


#### Set Hyperparameters (Extension)

We start with a first set of fixed hyperparameters. They will undergo changes along the way:

- *STEP_MEM* is the number of steps the agent should take into account as the current state it is in: This is the "operational" memory of the agent. I will refer to it as step memory.
- *NUM_HIDDEN_NEURON* is the number of neurons in the hidden layer, see below
- *INITIALIZATION* is one of the available weight initializations in Keras, see below

In [10]:
STEP_MEM = 1
NUM_HIDDEN_NEURON = 200
INITIALIZATION = 'uniform'
ACTIVATION = 'tanh'

#### Build Keras Model

When I started with building models, I naturally had to get aquainted with the Keras API semantics first. As indicated above, I want to start out with a shallow model version to understand fully the model mechanics, and leave it to a later stage to deepen the model.

Thus, I chose Keras' *Sequential* model architecture, creating a model consisting of 

- one fully-connected hidden layer (Keras terminology is *Dense*) plus an non-linear 'relu' Activation, as seen with Ben Lau; just a shot in the dark.

- the output layer with linear activation, which produces values for each of the possible actions given a certain state input.

TODO loss, metrics

In [11]:
def _create_network():
    model = Sequential()
    model.add(Dense(NUM_HIDDEN_NEURON, init=INITIALIZATION, input_shape=(STEP_MEM*NUM_INPUT,))) 
    model.add(Activation(ACTIVATION))
    model.add(Dense(NUM_ACTION, init=INITIALIZATION))
    model.compile(optimizer='rmsprop', loss='mse') # sgd(lr=self.learning_rate)
    return model

'''
def _create_network():
    model = Sequential()
    model.add(Dense(NUM_HIDDEN_NEURON, init=INITIALIZATION, input_shape=(STEP_MEM*NUM_INPUT,))) 
    model.add(Activation(ACTIVATION))
    model.add(Dense(NUM_ACTION, init=INITIALIZATION))
    model.add(Activation('softmax'))
    model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model
'''

"\ndef _create_network():\n    model = Sequential()\n    model.add(Dense(NUM_HIDDEN_NEURON, init=INITIALIZATION, input_shape=(STEP_MEM*NUM_INPUT,))) \n    model.add(Activation(ACTIVATION))\n    model.add(Dense(NUM_ACTION, init=INITIALIZATION))\n    model.add(Activation('softmax'))\n    model.compile(optimizer='rmsprop',\n                loss='categorical_crossentropy',\n                metrics=['accuracy'])\n    return model\n"

#### Build Training Epoch

In [12]:
def dqn_1(model, render=False):
    # Init epoch stats
    stats = {}

    # Play through episodes
    for episode in range(NUM_EPISODE): 
        # Init episode stats
        stats[episode] = [0]*3
        
        # Init observations
        x_t = ENV.reset()
        # Pile up FRAME_MEM times the same init observation, in order to be consistent with the model input
        s_t = np.tile(x_t, STEP_MEM)
        done = False

        # Start an episode
        for step in count():
            
            # Reset the environment for a new gaming step
            if render: ENV.render()
    
            # Estimate q for each action at s_t
            q = model.predict(s_t[np.newaxis])[0] 

            # Take action with highest estimated reward (argmax returns index)
            a_t = np.argmax(q) 
            
            # Observe after action a_t
            x_t, r_t, done, info = ENV.step(a_t) 
            
            # Create state at t1: Append x observations, throw away the earliest
            s_t1 = np.concatenate((x_t, s_t[:(STEP_MEM-1) * NUM_INPUT,]), axis=0) 

            # Estimate q for each action at s_t1 (again a forward pass)
            Q_sa = model.predict(s_t1[np.newaxis])[0] 

            ''' Create reference/targets by updating estimated reward for chosen action:
                For action taken, replace estimated reward by remaining cumulative lifetime reward
            ''' 
            targets = q
            targets[a_t] = r_t + GAMMA * np.max(Q_sa) if not done else r_t

            ''' Learn!
                - Again, predict q values for state s_t
                - Calculate loss by comparing predictions to targets: they will differ only for the action taken
                - backpropagate error for action taken, update weights
            ''' 
            model.fit(s_t[np.newaxis], targets[np.newaxis], verbose=0)
            
            # Update statistics: Sum up rewards, count steps, successes, solved
            stats[episode][0] += r_t
            if r_t >= 100: stats[episode][1] += 1
            if r_t >= 200: stats[episode][2] += 1
            
            # Visualize
            stdout.write("\rStep {} @ Episode {}/{}: Reward {}, Success {}, Solved {}".format(
                       step, episode+1, NUM_EPISODE, stats[episode][0], stats[episode][1], stats[episode][2], end=""))
   
            if done or step > 10000: break

            # Update state
            s_t = s_t1
            
    return stats

#### Train Model

Let's try:

In [13]:
model = _create_network() # Initialize model
stats = dqn_1(model, render=False) # Train model / Play epoch
print "\rMaximum Reward {}, Successful Episodes {}, Solved Episodes {}".format\
                    ( max([ v[0] for v in stats.values() ])\
                     , sum([ v[1] for v in stats.values() ])\
                     , sum([ v[2] for v in stats.values() ]))

Maximum Reward -33.0060862972, Successful Episodes 5, Solved Episodes 0


That is quite working nicely already! The frantic, purpose-less behaviour is gone most of the time, or is vanishing quickly within a few episodes only.

What can learn from this basic implementation?

- We could consider the model architecture:

    - Does the weight initializations make sense? What are the alternatives?
    - Does the nonlinear activation function for the hidden layer make sense? What are the alternatives? 
    - What about the number of neurons in the hidden layer?
    
We already step into the vast domain of parameter tuning for ANNs. 

We also should think of tuning the enviroment / model input. step memory *STEP_MEM*: Does it make sense to let the agent know only its current state, or shall we allow him to take into consideration also some of the states before? If yes, how far back should it remember? 

We will explore parameter tuning in depth at the end of this notebook.

#### Observations

- Swinging movement: Some configurations produce an extensive swinging movement. The agent tries to counter-act skewed positions by engaging the lateral engines, and overdoes it. Then, it corrects again, and again, and from all these corrections forgets to fire against the moon's gravity, and the agent is crashing into the surface.
- Local minima. There are plenty of times one can see the agent trapped into a locally optimal policy. For example, it stays on the ground, engaging left and right engine forever, perfectly stable, but not reaching the ultimate goal. Or a setting where left and right engines are engaged, but the lower engine does not fire at all, over long episodes.

### 2) *Epsilon Greedy* Deep Q Learning from Single Observations

At this point, let us tackle the issue of getting stuck in local minima. As a remedy, Deep Q Learning makes use the so called *epsilon greedy* action selection policy. It allows for a random move with probability *epsilon*, and by that introduces the notion of exploration (random moves) vs. exploitation (act on estimation of the q function). 

*Exploration* will reduce the probability of getting stuck in local minima, which are *not* reflecting the best action given a certain experience level of the agen. It's just like fresh air for the AI brain, introducing random ideas from outside. 

On the other hand, the agent needs to train and get experience with his selected moves. It needs evidence that one decision was (not) the right one, and to update its decicion finding process (the weights of the ANN. It only gets it by acting according to its own decisions undisturbed by random inputs. This is where *exploitation* comes in.

In RL, usually *epsilon* decreases over a certain exploration period. This reflects the idea that the agent will start with many random moves to fathom the environment by just observing. With time and growing experience, it will decrease the share of random moves, since it feels more confident in its own decisions. 

I will follow th custom of allowing random moves at a linearly decreasing exploration rate during the exploration period. Following Deep Mind: Playing Atari with Deep Reinforcement Learning (p.6), we define a interval between the maximum and minimum *epsilon* allowed (*EPSILON_RANGE*), starting with complete randomness, and ending with 10% random moves. The exploration period (*NUM_EXPLORATION_STEP*) is set to 1/10 of *NUM_EPISODE*.

During training, the agent will run on the minimum *epsilon* constantly.

#### Set Hyperparameters (Extension)

In [14]:
EPSILON_RANGE = [1, 0.01]
NUM_EXPLORATION_STEP = 1 if NUM_EPISODE < 10 else NUM_EPISODE / 10

#### Build Training Epoch

In [15]:
def dqn_2(model, render=False): 
    ##NEW Initialize epsilon at its maximum value
    epsilon = EPSILON_RANGE[0] 
    
    # Init epoch stats
    stats = {}

    # Play through episodes
    for episode in range(NUM_EPISODE): 
        # Init episode stats
        stats[episode] = [0]*3
        
        # Init observations        
        x_t = ENV.reset()
        s_t = np.tile(x_t, STEP_MEM)
        done = False

        # Start an episode
        for step in count():
            
            # Reset the environment for a new gaming step
            if render: ENV.render()
            
            ###NEW Anneal random exploration rate epsilon over exploration period
            epsilon = epsilon - epsilon / NUM_EXPLORATION_STEP if epsilon > EPSILON_RANGE[1] else EPSILON_RANGE[1]
    
            # Estimate q for each action at s_t
            q = model.predict(s_t[np.newaxis])[0]

            # Take action with highest estimated reward 
            ###NEW Do this with probability 1-epsilon ("epsilon greedy" policy)
            a_t = np.argmax(q) if np.random.random() > epsilon else np.random.choice(ACTIONS, 1)[0]

            # Observe after action a_t
            x_t, r_t, done, info = ENV.step(a_t)
        
            # Create state at t1: Append x observations, throw away the earliest
            s_t1 = np.concatenate((x_t, s_t[:(STEP_MEM-1) * NUM_INPUT,]), axis=0)

            # Estimate q for each action at s_t1 (again a forward pass)
            Q_sa = model.predict(s_t1[np.newaxis])[0]

            ''' Create reference/targets by updating estimated reward for chosen action
                For action taken, replace estimated reward by remaining cumulative lifetime reward
            ''' 
            targets = q
            targets[a_t] = r_t + GAMMA * np.max(Q_sa) if not done else r_t

            ''' Learn!
                - Again, predict q values for state s_t
                - Calculate loss by comparing predictions to targets: they will differ only for the action taken
                - backpropagate error for action taken, update weights
            ''' 
            model.fit(s_t[np.newaxis], targets[np.newaxis], verbose=0)
            
            # Update statistics: Sum up rewards, count steps, successes, solved
            stats[episode][0] += r_t
            if r_t >= 100: stats[episode][1] += 1
            if r_t >= 200: stats[episode][2] += 1
            
            # Visualize
            stdout.write("\rStep {} @ Episode {}/{}: Reward {}, Success {}, Solved {}".format(
                       step, episode+1, NUM_EPISODE, stats[episode][0], stats[episode][1], stats[episode][2], end=""))
   
            if done or step > 10000: break            

            # Update state
            s_t = s_t1
            
    return stats

#### Train Model

In [16]:
stats = dqn_2(model) # Train model / Play epoch
print "\rMaximum Reward {}, Successful Episodes {}, Solved Episodes {}".format\
                    ( max([ v[0] for v in stats.values() ])\
                     , sum([ v[1] for v in stats.values() ])\
                     , sum([ v[2] for v in stats.values() ]))

Maximum Reward -15.9481827836, Successful Episodes 2, Solved Episodes 0


### 3) Deep Q Learning *from Stored Experiences* 

Deep Mind: Playing Atari with Deep Reinforcement Learning claims that "learning directly from consecutive samples is inefficient, due to the strong correlations between the samples" (p.5). Instead, the trick is to learn from a memory storage, which I will call Experience Replay Memory (*ERM*), in batches. 

We are thus going to create the main database of the agent: It is the place where it 
- stores states and its experiences with the states (transitions *s_t*, *a_t*, *r_t*,and *s_t1*)
- recalls on the memory, collects a memory sample, trains on the sample, and updates the Q function estimator.

The *ERM* is set up once per epoch and is fed at each step with fresh transition evidence.

#### Namespace (Extension)

In [17]:
from collections import deque

#### Set Hyperparameters (Extension)

*ERM_SIZE* is setting the size of the experience replay memory. Following the recommendation of Deep Mind: Playing Atari with Deep Reinforcement Learning (p.6), we set it equal to the number of exploration steps. *BATCH_SIZE* denotes the size of the transition sample which is drawn uniformly without replacement from the *ERM* at each step. Again, its size is following the recommendations of Deep Mind: Playing Atari with Deep Reinforcement Learning (p.6).

In [18]:
ERM_SIZE = NUM_EXPLORATION_STEP
BATCH_SIZE = 32

#### Build Training Epoch

In [19]:
def dqn_3(model, render=False):
    # Init epsilon and ERM
    epsilon = EPSILON_RANGE[0]
    ERM = deque(maxlen=ERM_SIZE)
    
    # Init epoch stats
    stats = {}
    
    # Play through episodeszv7
    for episode in range(NUM_EPISODE): 
        # Init episode stats
        stats[episode] = [0]*3
        
        # Init observations        
        x_t = ENV.reset()
        s_t = np.tile(x_t, STEP_MEM)
        done = False

        # Start an episode
        for step in count():
            
            # Reset the environment for a new gaming step
            if render: ENV.render()
            
            # Anneal random exploration rate epsilon over exploration period
            epsilon = epsilon - epsilon / NUM_EXPLORATION_STEP if epsilon > EPSILON_RANGE[1] else EPSILON_RANGE[1]
    
            # Estimate q for each action at s_t
            q = model.predict(s_t[np.newaxis])[0]

            # Take action with highest estimated reward with probability 1-epsilon ("epsilon greedy" policy)
            a_t = np.argmax(q) if np.random.random() > epsilon else np.random.choice(ACTIONS, 1)[0]

            # Observe after action a_t
            x_t, r_t, done, info = ENV.step(a_t)
        
            # Create state at t1: Append x observations, throw away the earliest
            s_t1 = np.concatenate((x_t, s_t[:(STEP_MEM-1) * NUM_INPUT,]), axis=0)
            
            ###NEW Store transition in experience replay memory
            ERM.append((s_t, a_t, r_t, s_t1))

            ###NEW Choose a batch of maximum length BATCH_SIZE
            minibatch = np.array([ ERM[i] for i in np.random.choice(np.arange(0, len(ERM)), min(len(ERM), BATCH_SIZE)) ])
            
            ###NEW Compute targets/reference for each transition in minibatch
            inputs = deque(); targets = deque()
            for m in minibatch:
                inputs.append(m[0]) # Append s_t of batch transition m to inputs
                m_q = model.predict(m[0][np.newaxis])[0] # Estimate rewards for each action (targets), at s_t
                m_Q_sa = model.predict(m[3][np.newaxis])[0] # Estimate rewards for each action (targets), at s_t1
                m_targets = m_q
                m_targets[m[1]] = m[2] + GAMMA * np.max(m_Q_sa)
                targets.append(m_targets) # Append target of batch transition m to targets
                
            ###NEW Train the model by backpropagating the errors and update weights
            model.train_on_batch(np.array(inputs), np.array(targets))
            
            # Update statistics: Sum up rewards, count steps, successes, solved
            stats[episode][0] += r_t
            if r_t >= 100: stats[episode][1] += 1
            if r_t >= 200: stats[episode][2] += 1
            
            # Visualize
            stdout.write("\rStep {} @ Episode {}/{}: Reward {}, Success {}, Solved {}".format(
                       step, episode+1, NUM_EPISODE, stats[episode][0], stats[episode][1], stats[episode][2], end=""))
            
            if done or step > 10000: break           
            
            # Update state
            s_t = s_t1  
            
    return stats

#### Train Model

In [20]:
stats = dqn_3(model) # Train model / Play epoch
print "\rMaximum Reward {}, Successful Episodes {}, Solved Episodes {}".format\
                    ( max([ v[0] for v in stats.values() ])\
                     , sum([ v[1] for v in stats.values() ])\
                     , sum([ v[2] for v in stats.values() ]))

Maximum Reward -4.15058880697, Successful Episodes 2, Solved Episodes 0


#### Observations

The question crossed my mind: Why don't we predict beforehand, on the fly, at every step? Would that not be computationally efficient? This has the huge disadvantage that we predict with the knowledge available at timestep t. This might be faulty, and the faulty prediction stays as target reference in the batch, and is used to compare the loss for the taken action between the prediction at timestep t and the potenitially long ago target estimation. This will bias the learning process significantly. Thus, we select a batch, calculate the estimations and targets for the complete batch, both with the knowledge of the current steps.

### 4) Deep Q Learning from Stored Experiences, *Brute Force Trials*

At this point, we will finalize training by applying some more tweaks, outside and within the model block. The goal is to find the best parameter combination by brute force. We install some extra bookeeping in order to catch the most successful epochs.

Most importantly, we establish two new parameters:

- Steps per Action (*SPA*) determines how many steps pass before a fresh action is taken. Deep Mind - Playing Atari with Deep Reinforcement Learning suggests 4 skipping 4 time steps before conducting an action (p.6).
- Reward Clipping (*R_CLIP*) is also an idea taken from Deep Mind - Playing Atari with Deep Reinforcement Learning (p.6), where it was implemented in order to make rewards comparable across the different games.

Besides that, we collect statistics:

- The reward sum per episode
- A flag if the lander was successful during the episode (with a reward greater or equal 100)
- A flag if the game was solved (with a reward greater or equal 200) TODO

Finally, we establish a parameter testing routine, which will go through parameter settings with brute force. If the epoch training led to at least one success, the epoch is stored with its id and the three above-mentioned statistics. The goal is to identify the well-working settings, in order to run them afterwards on a intense scale again.

For this step, I will switch to AWS EC2. I make my first steps on Ubuntu 16.04 t2.micro free tier instances, and then switch to a Ubuntu 14.04 g2.2xlarge instance. This notebook now runs on a notebook server. It took me a while to get aquainted with the AWS UX, to set up an IAM user, a Security Group, all the dependencies for this project, and the notebook server, but of course the speed at hand makes up for it easily. 

#### Namespace (Extension)

In [21]:
import json
from keras.models import model_from_json
from os import path, makedirs

#### Set Hyperparameters (Extension)

- Step Memory *STEP_MEM*. We replace the fixed hyperparameter by an arbitrary range of *STEP_MEM* candidates (*STEP_MEM_RANGE*)
- Keras' available initializations *INITIALIZATIONS*, for all layers
- Keras' available activations *ACTIVATIONS*
- Discount factor *GAMMA*. We replace the fixed hyperparameter by an arbitrary range of candidates *GAMMA_RANGE*.
- Take action every n-th step (step per action *SPA_RANGE*)
- Reward Clipping (*R_CLIP*)
- *SAVE_PATH* is the working directory where the training epoch models are stored

In [22]:
NUM_EPISODE = 10**3

STEP_MEM_RANGE = np.arange(1, 31) # Constant over one epoch
INITIALIZATIONS = ['uniform', 'lecun_uniform', 'normal', 'identity', 'orthogonal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'] # Constant over one epoch
ACTIVATIONS = ['softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'] # Constant over one epoch
GAMMA_RANGE = np.arange(0.0, 1.01, 0.01) # Constant over one epoch
SPA_RANGE = np.arange(1, 5) # Constant over one epoch
R_CLIP = [False, True] # Constant over one epoch

SAVE_PATH = path.join(path.expanduser("~"), "RL_Saves")
try: 
    makedirs(SAVE_PATH)
except OSError:
    if not path.isdir(SAVE_PATH):
        raise

#### Build Training Epoch

In [ ]:
def dqn_4(model, render=False):
    # Init epsilon and ERM
    epsilon = EPSILON_RANGE[0]
    ERM = deque(maxlen=ERM_SIZE)
    
    # Init epoch stats
    stats = {}

    # Play through episodes
    for episode in range(NUM_EPISODE): 
        # Init episode stats
        stats[episode] = [0]*3
        
        # Init observations        
        x_t = ENV.reset()
        s_t = np.tile(x_t, STEP_MEM)
        done = False

        # Start an episode
        for step in count():
            
            # Reset the environment for a new gaming step
            if render: ENV.render()
            
            # Anneal random exploration rate epsilon over exploration period
            epsilon = epsilon - epsilon / NUM_EXPLORATION_STEP if epsilon > EPSILON_RANGE[1] else EPSILON_RANGE[1]
        
            # Estimate q for each action at s_t
            q = model.predict(s_t[np.newaxis])[0]

            # Take action with highest estimated reward with probability 1-epsilon ("epsilon greedy" policy)
            ###NEW Act only every n-th step
            if episode % SPA == 0: a_t = np.argmax(q) if np.random.random() > epsilon else np.random.choice(ACTIONS, 1)[0]

            # Observe after action a_t
            x_t, r_t, done, info = ENV.step(a_t)
            
            # Create state at t1: Append x observations, throw away the earliest
            s_t1 = np.concatenate((x_t, s_t[:(STEP_MEM-1) * NUM_INPUT,]), axis=0)
            
            ###NEW Clip rewards
            if R_CLIP and r_t != 0: r_t = abs(r_t) / r_t
            
            # Store transition in experience replay memory
            ERM.append((s_t, a_t, r_t, s_t1))

            # Choose a batch of maximum length BATCH_SIZE
            minibatch = np.array([ ERM[i] for i in np.random.choice(np.arange(0, len(ERM)), min(len(ERM), BATCH_SIZE)) ])
            
            # Compute targets/reference for each transition in minibatch
            inputs = deque(); targets = deque()
            for m in minibatch:
                inputs.append(m[0]) # Append s_t of batch transition m to inputs
                m_q = model.predict(m[0][np.newaxis])[0] # Estimate rewards for each action (targets), at s_t
                m_Q_sa = model.predict(m[3][np.newaxis])[0] # Estimate rewards for each action (targets), at s_t1
                m_targets = m_q
                m_targets[m[1]] = m[2] + GAMMA * np.max(m_Q_sa)
                targets.append(m_targets) # Append target of batch transition m to targets
                
            # Train the model by backpropagating the errors and update weights
            model.train_on_batch(np.array(inputs), np.array(targets))
                    
            # Update statistics: Sum up rewards, count steps, successes, solved
            stats[episode][0] += r_t
            if r_t >= 100: stats[episode][1] += 1
            if r_t >= 200: stats[episode][2] += 1
             
            if done or step > 10000: break
            
            # Update state
            s_t = s_t1
    
    return stats

#### Train Model

In [ ]:
# Init epoch overview
epoch_account = {}

# Apply brute force
for ALPHA in ALPHA_RANGE:
    for STEP_MEM in STEP_MEM_RANGE:
        for NUM_HIDDEN_NEURON in NUM_HIDDEN_NEURON_RANGE:
            for INITIALIZATION in INITIALIZATIONS:
                for ACTIVATION in ACTIVATIONS:
                    # Initialize model
                    try: model = _create_network() 
                    except Exception: continue
                    
                    for GAMMA in GAMMA_RANGE: 
                        for SPA in SPA_RANGE:
                            for R_C in R_CLIP:
                                # Create model id
                                EPOCH = '_'.join([repr(ALPHA), repr(STEP_MEM), str(NUM_HIDDEN_NEURON), INITIALIZATION, ACTIVATION, str(round(GAMMA, 2)), repr(SPA), repr(R_C)[:1]])
                                
                                # Train model / Play epoch
                                stats = dqn_4(model) 
                                
                                # Calculate epoch stats
                                highest_reward = max([ v[0] for v in stats.values() ])
                                success_episodes = sum([ v[1] for v in stats.values() ])
                                solved_episodes = sum([ v[2] for v in stats.values() ])
                                
                                # Visualize
                                stdout.write("\rEpoch {}, Maximum Reward {}, Successful Episodes {}, Solved Episodes {}".format(\
                                     EPOCH, highest_reward, success_episodes, solved_episodes))
                                
                                # Memorize
                                if success_episodes > 0: 
                                    epoch_account[EPOCH] = [highest_reward, success_episodes, solved_episodes]

# Write out
with open(path.join(SAVE_PATH, "DQN_Stats.json"), "w") as outfile: json.dump(epoch_account, outfile) 

In [ ]:
'''
###NEW Set up model, weight, and stats save
    MH5 = path.join(SAVE_PATH, "DQN_"+EPOCH+".h5")
    MJS = path.join(SAVE_PATH, "DQN_"+EPOCH+".json") 
    
        # Save model and model weights
        with open(MJS, "w") as outfile: json.dump(model.to_json(), outfile) 
        model.save_weights(MH5_V, overwrite=True)
'''

In [ ]:
'''
    if args['mode'] == 'Run':
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=1e-6)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")  
'''